In [1]:
!pip install xlrd
!pip install geopandas
!pip install openpyxl

  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Using cached geopandas-0.11.0-py3-none-any.whl (1.0 MB)
  Using cached pyproj-3.3.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.7 MB)
  Using cached Fiona-1.8.21-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.6 MB)
  Using cached Shapely-1.8.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (2.1 MB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [2]:
import pandas as pd
import folium  #needed for interactive map
from folium import IFrame
from folium.plugins import MarkerCluster 
import geopandas as gpd
#import shapefile as shp
from folium import FeatureGroup, LayerControl

In [3]:
df_turistic_places=pd.read_csv('7. Base de Atractivos turísticos (2021).csv',encoding='latin1')

In [4]:
df_turistic_places['color']=df_turistic_places['Tipo de Patrimonio'].replace({'Patrimonio cultural material inmueble':'lightblue',
                                                                             'Patrimonio cultural inmaterial':'lightred',
                                                                              'Áreas protegidas':'orange',
                                                                              'Patrimonio Cultural Mueble':'green',
                                                                              'Aguas lénticas':'pink',
                                                                              'Montañas':'lightgreen',
                                                                              'Aguas lóticas':'lightgray',
                                                                              'Formaciones cársicas':'gray'
                                                                              }
                                                                            )
#https://matplotlib.org/3.5.0/gallery/color/named_colors.html página de colores

In [5]:
#Define Icons

df_turistic_places['icon']=df_turistic_places['color']
#https://matplotlib.org/3.5.0/gallery/color/named_colors.html página de colores

for i in range(0,len(df_turistic_places)):
    if df_turistic_places.iloc[i]['Tipo de Patrimonio']=='Patrimonio cultural material inmueble':
        df_turistic_places['icon'].iloc[i]="globe"
        
    if df_turistic_places.iloc[i]['Tipo de Patrimonio']=='Patrimonio cultural inmaterial':
        df_turistic_places['icon'].iloc[i]="paint-brush"
        
    if df_turistic_places.iloc[i]['Tipo de Patrimonio']=='Áreas protegidas':
        df_turistic_places['icon'].iloc[i]="tree"
        
    if df_turistic_places.iloc[i]['Tipo de Patrimonio']=='Patrimonio Cultural Mueble':
        df_turistic_places['icon'].iloc[i]="institution"
        
    if df_turistic_places.iloc[i]['Tipo de Patrimonio']=='Aguas lénticas':
        df_turistic_places['icon'].iloc[i]="pagelines"
        
    if df_turistic_places.iloc[i]['Tipo de Patrimonio']=='Montañas':
        df_turistic_places['icon'].iloc[i]="toggle-up"
        
    if df_turistic_places.iloc[i]['Tipo de Patrimonio']=='Aguas lóticas':
        df_turistic_places['icon'].iloc[i]="life-bouy"
        
    if df_turistic_places.iloc[i]['Tipo de Patrimonio']=='Formaciones cársicas':
        df_turistic_places['icon'].iloc[i]="sun-o"
#https://fontawesome.com/v4/icons/

/home/jovyan/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
poligonos=gpd.read_file('/home/jovyan/DataBases-Estefania/poligonos-localidades.shp',encoding='UTF-8')
#poligonos
#https://bogota-laburbano.opendatasoft.com/explore/dataset/poligonos-localidades/export/?flg=es

In [15]:
#Crear Mapa
map = folium.Map(location=[4.60971,-74.08175], tiles='CartoDB positron')

lista=range(1,poligonos.shape[0]+1)
j=3

#Mostrar Polígono
for _, r in poligonos.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson (data=geo_j,
                           style_function=lambda x: {'fillColor': 'white'})
    folium.Popup(r['Nombre_de_l']).add_to(geo_j)
    geo_j.add_to(map)
    j=j+1
    
#Puntos Turísticos
i=0
for _, r in df_turistic_places.iterrows():
    html=f"""
    <h1> {df_turistic_places.iloc[i]['Tipo de Patrimonio']}</h1>
    <p></p>
    {df_turistic_places.iloc[i]['Nombre']}
    <li>{df_turistic_places.iloc[i]['Direccion Atractivo']}</li>
    </ul>
    </p>
    """
    icon=df_turistic_places.iloc[i]['icon']
    color=df_turistic_places.iloc[i]['color']
    
    lat = r['Latitud']
    lon = r['Longitud']
    folium.Marker(location=[lat, lon],
                  popup=html,
                  icon=folium.Icon(color=color, icon=icon,prefix='fa'),
                 ).add_to(map)
    i=i+1
map

In [16]:
map.save('LugaresTurísticos.html')